# 1 Exercise 2

## 2.1 - TITLE
Description

## 2.2 Import dependencies

In [1]:
import datetime
import numpy as np
import scipy.stats as sts
import matplotlib.pyplot as plt

from scipy.integrate import quad
from typing import Tuple, List

## 2.3 Auxiliary Functions

Function to calculate optimal percentage. Note that $F^{-1}_{Y}(\frac{\tilde{p}}{\tilde{p} + \tilde{c}}) = F^{-1}_Y(\frac{p - c}{p + c_h})$.

In [2]:
def optimal_percentage(price: float, cost: float, holding_cost: float) -> float:
    return (price - cost)/(price + holding_cost)

Function that calculates the optimal quantity of a normal distribution.

In [3]:
def calculate_known_optimal_quantity_normal(cost: float, price: float, holding_cost: float, location: float, scale: float) -> float:
    optimal_percent: float = optimal_percentage(price, cost, holding_cost)
    return sts.norm.ppf(optimal_percent, loc=location, scale=scale)

def calculate_known_optimal_quantity_lognormal(cost: float, price: float, holding_cost: float, mu: float, sigma: float) -> float:
    optimal_percent: float = optimal_percentage(price, cost, holding_cost)
    return sts.lognorm.ppf(optimal_percent, sigma, scale=np.exp(mu))

Function that calculates the mean squared error.

In [4]:
def root_mean_squared_error(quantity_estimation_vector: np.ndarray, known_optimal_quantity: float) -> float:
    estimation_error: np.ndarray = quantity_estimation_vector - known_optimal_quantity
    squared_error: float = np.square(estimation_error).sum()

    vector_size: int = quantity_estimation_vector.size
    mean_squared_error: float = squared_error/vector_size
    return np.sqrt(mean_squared_error)


Function that calculates the profit loss ratio.

In [5]:
def profit_loss_ratio(profit_estimation_vecor: np.ndarray, known_maximum_profit: float) -> float:
    ratio: np.ndarray = (known_maximum_profit - profit_estimation_vecor)/known_maximum_profit
    vector_size: int = profit_estimation_vecor.size
    return np.absolute(ratio).sum()/vector_size


Function that estimates the parameters from a normal distribution, function that calculates parametric optimal quantity.

In [6]:
def normal_parametric_estimation(data_vector: np.ndarray) -> Tuple[float, float]:
    mean: float = data_vector.mean()
    var: float = data_vector.var()
    return (mean, np.sqrt(var))

def parametric_normal_optimal_quantity(cost: float, holding_cost: float, price: float, data_vector: np.ndarray) -> float:
    mean, std = normal_parametric_estimation(data_vector)
    optimal_percent: float = optimal_percentage(price, cost, holding_cost)
    inv_cdf_quantity: float =  sts.norm.ppf(optimal_percent, loc=mean, scale=std)

    return inv_cdf_quantity

Function that estimates parameters from a lognormal distribution, function that calculates optimal quantity of a lognormal distriution.

In [7]:
def log_normal_parametric_estimation(data_vector: np.ndarray) -> Tuple[float, float]:
    # Dit moet nog even gecheckt worden, ik heb geen idee of dit is hoe je het berekent. Micha
    mean: float = np.mean(np.log(data_vector))
    var: float = np.var(np.log(data_vector))
    return (mean, np.sqrt(var))

def parametric_lognormal_optimal_quantity(cost: float, holding_cost: float, price: float, data_vector: np.ndarray) -> float:
    mean, std = log_normal_parametric_estimation(data_vector)
    optimal_percent: float = optimal_percentage(price, cost, holding_cost)
    inv_cdf_quantity: float = sts.lognorm.ppf(optimal_percent, std, scale=np.exp(mean))

    return inv_cdf_quantity

Function that calculates the non parametric optimal quantity.

In [8]:
def non_parametric_optimal_quantity(cost: float, holding_cost: float, price: float, data_vector: np.ndarray) -> float:
    sorted_data_vector: np.ndarray = data_vector.sort()
    vector_size = data_vector.size
    optimal_percent = optimal_percentage(price, cost, holding_cost)
    optimal_arg_value = int(np.ceil(optimal_percent * vector_size))

    return data_vector[optimal_arg_value - 1]

In [9]:
def normal_monte_carlo_simulation_and_paramatric_estimation(number: float, target_surface: float, location: float, scale: float, price: float) -> float:
    data_vector: np.ndarray = np.random.normal(loc=location, scale=scale, size=number)
    return parametric_normal_optimal_quantity(1 - target_surface, 0, price, data_vector)


In [10]:
def lognormal_monte_carlo_simulation_and_paramatric_estimation(number: float, target_surface: float, mu: float, sigma: float, price: float) -> float:
    data_vector: np.ndarray = np.random.lognormal(mu, sigma, size=number)
    return parametric_lognormal_optimal_quantity(1 - target_surface, 0, price, data_vector)

In [11]:
def normal_monte_carlo_simulation_and_non_paramatric_estimation(number: float, target_surface: float, location: float, scale: float, price: float) -> float:
    data_vector: np.ndarray = np.random.normal(loc=location, scale=scale, size=number)
    return non_parametric_optimal_quantity(1 - target_surface, 0, price, data_vector)

def lognormal_monte_carlo_simulation_and_non_paramatric_estimation(number: float, target_surface: float, mu: float, sigma: float, price: float) -> float:
    data_vector: np.ndarray = np.random.lognormal(mu, sigma, size=number)
    return non_parametric_optimal_quantity(1 - target_surface, 0, price, data_vector)

In [12]:
def calculate_normal_expected_profit(location_known: float, scale_known: float, price: float, cost: float, quantity: float) -> float:
    integral = quad(lambda y: sts.norm.cdf(y, loc=location_known, scale=scale_known), -np.inf, quantity)[0]
    return (price - cost) * quantity - price * integral

def calculate_lognormal_expected_profit(location_known: float, std_known: float, price: float, cost: float, quantity: float) -> float:
    integral = quad(lambda y: sts.lognorm.cdf(y, std_known, loc=np.exp(location_known)), -np.inf, quantity)[0]
    return (price - cost) * quantity - price * integral

In [13]:
def calculate_normal_parametric_metrics(number: int, target_surface: float, scale: float, price: float, estimations: int, location: float) -> Tuple[float, float]:
    quantity_estimation_results: np.ndarray = np.empty(estimations)
    profit_estimation_results: np.ndarray = np.empty(estimations)

    for i in range(estimations):
        quantity_estimation_results[i] = normal_monte_carlo_simulation_and_paramatric_estimation(number, target_surface, location, scale, price)
        profit_estimation_results[i] = calculate_normal_expected_profit(location, scale, price, 1 - target_surface, quantity_estimation_results[i])

    known_optimal_quantity: float = calculate_known_optimal_quantity_normal(1 - target_surface, price, 0, location, scale)
    known_maximum_profit: float = calculate_normal_expected_profit(location, scale, price, 1 - target_surface, known_optimal_quantity)
    rsme: float = root_mean_squared_error(quantity_estimation_results, known_optimal_quantity)
    # Calculate PLR
    plr: float = profit_loss_ratio(profit_estimation_results, known_maximum_profit)
    return (rsme, plr)
    

In [14]:
def calculate_lognormal_parametric_metrics(number: int, target_surface: float, scale: float, price: float, estimations: int, location: float) -> Tuple[float, float]:
    quantity_estimation_results: np.ndarray = np.empty(estimations)
    profit_estimation_results: np.ndarray = np.empty(estimations)

    for i in range(estimations):
        quantity_estimation_results[i] = lognormal_monte_carlo_simulation_and_paramatric_estimation(number, target_surface, location, scale, price)
        #print(f"quantity: {quantity_estimation_results[i]}")
        profit_estimation_results[i] = calculate_lognormal_expected_profit(location, scale, price, 1 - target_surface, quantity_estimation_results[i])
        #print(f"profit: {profit_estimation_results[i]}")

    known_optimal_quantity: float = calculate_known_optimal_quantity_lognormal(1 - target_surface, price, 0, location, scale)
    known_maximum_profit: float = calculate_lognormal_expected_profit(location, scale, price, 1 - target_surface, known_optimal_quantity)
    rsme: float = root_mean_squared_error(quantity_estimation_results, known_optimal_quantity)
    plr: float = profit_loss_ratio(profit_estimation_results, known_maximum_profit)
    return (rsme, plr)

In [15]:
def calculate_normal_non_parametric_metrics(number: int, target_surface: float, scale: float, price: float, estimations: int, location: float):
    quantity_estimation_results: np.ndarray = np.empty(estimations)
    profit_estimation_results: np.ndarray = np.empty(estimations)


    for i in range(estimations):
        quantity_estimation_results[i] = normal_monte_carlo_simulation_and_non_paramatric_estimation(number, target_surface, location, scale, price)
        profit_estimation_results[i] = calculate_normal_expected_profit(location, scale, price, 1 - target_surface, quantity_estimation_results[i])

    known_optimal_quantity: float = calculate_known_optimal_quantity_normal(1 - target_surface, price, 0, location, scale)
    known_maximum_profit: float = calculate_normal_expected_profit(location, scale, price, 1 - target_surface, known_optimal_quantity)
    rsme: float = root_mean_squared_error(quantity_estimation_results, known_optimal_quantity)
    plr: float = profit_loss_ratio(profit_estimation_results, known_maximum_profit) # Calculate
    return (rsme, plr)

def calculate_lognormal_non_parametric_metrics(number: int, target_surface: float, scale: float, price: float, estimations: int, location: float):
    quantity_estimation_results: np.ndarray = np.empty(estimations)
    profit_estimation_results: np.ndarray = np.empty(estimations)


    for i in range(estimations):
        quantity_estimation_results[i] = lognormal_monte_carlo_simulation_and_non_paramatric_estimation(number, target_surface, location, scale, price)
        profit_estimation_results[i] = calculate_lognormal_expected_profit(location, scale, price, 1 - target_surface, quantity_estimation_results[i])

    known_optimal_quantity: float = calculate_known_optimal_quantity_lognormal(1 - target_surface, price, 0, location, scale)
    known_maximum_profit: float = calculate_lognormal_expected_profit(location, scale, price, 1 - target_surface, known_optimal_quantity)
    rsme: float = root_mean_squared_error(quantity_estimation_results, known_optimal_quantity)
    plr: float = profit_loss_ratio(profit_estimation_results, known_maximum_profit) # Calculate
    return (rsme, plr)

## 2.4 Title

In [16]:
def normal():
    date: str = datetime.datetime.utcnow().strftime("%y%m%d-%H%M%S")
    locations: List[int] = [50, 100, 200, 500]
    scales: List[int] = [1, 5, 10, 20]
    estimations: int = 1000
    number: List[int] = [10, 50, 100, 200]
    target_surface: List[int] = [0.01, 0.05, 0.1, 0.3, 0.5, 0.7, 0.9, 0.95, 0.99]
    price: int = 1

    output_data = np.zeros((len(locations), len(scales), len(number), len(target_surface), 2, 2))

    for a, location in enumerate(locations):
        for b, scale in enumerate(scales):
            for c, n in enumerate(number):
                for d, t in enumerate(target_surface):
                    print(f"n: {n}, t: {t}, loc: {location}, scale: {scale}")
                    parametric_metrics = calculate_normal_parametric_metrics(n, t, scale, price, estimations, location)
                    nonparametric_metrics = calculate_normal_non_parametric_metrics(n, t, scale, price, estimations, location)
                    print(f"parametric rsme: {parametric_metrics[0]}, plr: {parametric_metrics[1]}")
                    print(f"nonparametric rsme: {nonparametric_metrics[0]}, plr {nonparametric_metrics[1]}")
                    print("")

                    output_data[a][b][c][d] = np.array([parametric_metrics, nonparametric_metrics])

                    with open(f"r-norm-{date}.npy", "wb") as file:
                        np.save(file, output_data) # Save on every run because process might crash

In [17]:
print("start")
normal()
print("Finished")

start
Finished


In [18]:
def lognormal():
    date: str = datetime.datetime.utcnow().strftime("%y%m%d-%H%M%S")
    locations: List[int] = [4, 5, 6, 7, 8]#[50, 100, 200, 500]
    scales: List[int] = [0.2, 0.5, 0.8, 1] #[1, 5, 10, 20]
    estimations: int = 1000
    number: List[int] = [10, 50, 100, 200]
    target_surface: List[int] = [0.01, 0.05, 0.1, 0.3, 0.5, 0.7, 0.9, 0.95, 0.99]
    price: int = 1

    output_data = np.zeros((len(locations), len(scales), len(number), len(target_surface), 2, 2))

    for a, location in enumerate(locations):
        for b, scale in enumerate(scales):
            for c, n in enumerate(number):
                for d, t in enumerate(target_surface):
                    print(f"n: {n}, t: {t}, loc: {location}, scale: {scale}")
                    parametric_metrics = calculate_lognormal_parametric_metrics(n, t, scale, price, estimations, location)
                    nonparametric_metrics = calculate_lognormal_non_parametric_metrics(n, t, scale, price, estimations, location)
                    print(f"parametric rsme: {parametric_metrics[0]}, plr: {parametric_metrics[1]}")
                    print(f"nonparametric rsme: {nonparametric_metrics[0]}, plr {nonparametric_metrics[1]}")
                    print("")

                    output_data[a][b][c][d] = np.array([parametric_metrics, nonparametric_metrics])

                    with open(f"r-lognorm-{date}.npy", "wb") as file:
                        np.save(file, output_data) # Save on every run because process might crash

In [19]:
print("start")
lognormal()
print("finished")

n: 10, t: 0.01, loc: 4, scale: 0.2
parametric rsme: 4.647825522322858, plr: 0.10653666729047549
nonparametric rsme: 7.545759273616624, plr 0.18821523008253094

n: 10, t: 0.05, loc: 4, scale: 0.2
parametric rsme: 4.047056051384399, plr: 0.08160206047779238
nonparametric rsme: 4.771689439868607, plr 0.09704338328974042

n: 10, t: 0.1, loc: 4, scale: 0.2
parametric rsme: 3.833234824458306, plr: 0.07141332579274967
nonparametric rsme: 4.670371716477996, plr 0.08796919327048819

n: 10, t: 0.3, loc: 4, scale: 0.2
parametric rsme: 3.378553996149411, plr: 0.050610760719599863
nonparametric rsme: 4.277333139631523, plr 0.05967235816271083

n: 10, t: 0.5, loc: 4, scale: 0.2
parametric rsme: 3.587201608419408, plr: 0.038929127906229745
nonparametric rsme: 4.292135318606431, plr 0.054041584522249754

n: 10, t: 0.7, loc: 4, scale: 0.2


KeyboardInterrupt: 